📌 何をしていたか？ (目的)

01 で収集したデータ論文を**引用している側の論文（＝被引用論文）**の情報を収集し、全文 XML データをダウンロードしていました。

これは、実験で「どのセクションで引用されたか」を分析するための、最も重要なデータ収集ステップです。

✅ どこまで行っているか？ (達成状況)

data_papers.csv を読み込み、引用されている可能性が高い「被引用数 10 以上」のデータ論文を処理対象として選びました。

各データ論文について、Scopus API を使い、それを引用している論文のメタデータ（DOI, タイトルなど）をリストアップしました (ステップ A)。

リストアップした全被引用論文に対し、並列処理を用いて高速に ScienceDirect API にアクセスし、全文 XML のダウンロードを試みました (ステップ B)。

ダウンロードの成否（成功、キャッシュ、404 エラー、429 エラーなど）を記録し、最終的な結果を citing_papers_with_paths.csv に保存済みです。

429（レートリミット）エラーで失敗した論文を対象に、再試行するスクリプトも準備しました。

➡️ このノートブックの成果物:

citing_papers_with_paths.csv: 被引用論文のメタデータと、ダウンロードした XML へのパスや成否ステータスが記録された索引ファイル。

data/raw/fulltext/内の XML ファイル群: 実際にダウンロードされた本文データ。


In [1]:
import requests
import pandas as pd
import time
import os
from tqdm.notebook import tqdm
from urllib.parse import urlparse, parse_qs
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- 設定項目 ---
API_KEY = "90469972beed34fa2913dc1ad6a644ac" 
INPUT_DATA_PAPERS_CSV = '../data/processed/data_papers.csv'
OUTPUT_CITING_PAPERS_CSV = '../data/processed/citing_papers_raw.csv'
XML_OUTPUT_DIR = '../data/raw/xml/'
SEARCH_API_URL = "https://api.elsevier.com/content/search/scopus"
FULLTEXT_API_URL = "https://api.elsevier.com/content/article/eid/"

# --- 1. 入力データの読み込みと準備 ---
try:
    df_data_papers = pd.read_csv(INPUT_DATA_PAPERS_CSV)
    df_target = df_data_papers.dropna(subset=['eid'])
    df_target['citedby_count'] = pd.to_numeric(df_target['citedby_count'], errors='coerce').fillna(0)
    df_target = df_target[df_target['citedby_count'] >= 10].copy()
    df_target = df_target.sort_values(by='citedby_count', ascending=False).reset_index(drop=True)
    print(f"処理対象のデータ論文（被引用数10以上）: {len(df_target)}件")
except FileNotFoundError:
    print(f"エラー: '{INPUT_DATA_PAPERS_CSV}' が見つかりません。`01`のノートブックを先に実行してください。")
    df_target = pd.DataFrame()

# --- ステップA: ダウンロード対象となる全被引用論文のリストアップ ---
tasks = []
if not df_target.empty:
    print("\n[ステップA] ダウンロード対象の被引用論文リストを作成しています...")
    
    # データ論文を一つずつ処理
    for index, data_paper in tqdm(df_target.iterrows(), total=len(df_target), desc="データ論文をスキャン中"):
        data_paper_eid = data_paper['eid']
        data_paper_title = data_paper['title']
        
        # Scopus Search APIで被引用論文を検索
        search_params = {
            'apiKey': API_KEY, 'query': f"REF({data_paper_eid})",
            'cursor': '*', 'count': 25, 'view': 'STANDARD'
        }
        
        # カーソルを使ったページネーションループ
        while 'cursor' in search_params and search_params['cursor']:
            try:
                search_response = requests.get(SEARCH_API_URL, params=search_params)
                search_response.raise_for_status()
                search_data = search_response.json()
                
                entries = search_data.get('search-results', {}).get('entry', [])
                if not entries:
                    break
                
                # 収集した被引用論文のメタデータをtasksリストに追加
                for entry in entries:
                    tasks.append({
                        'citing_paper_eid': entry.get('eid'),
                        'citing_paper_doi': entry.get('prism:doi'),
                        'citing_paper_title': entry.get('dc:title'),
                        'citing_paper_year': entry.get('prism:coverDate', '')[:4],
                        'cited_data_paper_title': data_paper_title,
                    })

                # 次のカーソル情報を探して更新
                next_cursor_url = next((link.get('@href') for link in search_data.get('search-results', {}).get('link', []) if link.get('@ref') == 'next'), None)
                
                if next_cursor_url:
                    parsed_url = urlparse(next_cursor_url)
                    query_params = parse_qs(parsed_url.query)
                    search_params['cursor'] = query_params.get('cursor', [None])[0]
                else:
                    break # 次のページがなければこのデータ論文の処理は終了
                
                time.sleep(1)

            except requests.exceptions.RequestException as e:
                print(f"  - API検索中にエラー (EID: {data_paper_eid}): {e}")
                break # エラーが発生したら次のデータ論文へ
    
    print(f"[ステップA] 完了。合計 {len(tasks)} 件の被引用論文をリストアップしました。")


# --- ステップB: 本文XMLの並列ダウンロード ---

# 個々のXMLをダウンロードする関数
def download_xml(task):
    eid = task.get('citing_paper_eid')
    if not eid:
        return eid, None
        
    xml_path = os.path.join(XML_OUTPUT_DIR, f"{eid}.xml")
    
    if os.path.exists(xml_path):
        return eid, xml_path

    try:
        url = f"{FULLTEXT_API_URL}{eid}?apiKey={API_KEY}"
        response = requests.get(url, headers={'Accept': 'application/xml'}, timeout=30)
        if response.status_code == 200:
            with open(xml_path, 'w', encoding='utf-8') as f:
                f.write(response.text)
            return eid, xml_path
    except requests.exceptions.RequestException:
        pass
        
    return eid, None

# マルチスレッドで並列実行
updated_tasks = []
if tasks:
    os.makedirs(XML_OUTPUT_DIR, exist_ok=True)
    print(f"\n[ステップB] {len(tasks)}件の論文XMLのダウンロードを並列で開始します...")
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_eid = {executor.submit(download_xml, task): task['citing_paper_eid'] for task in tasks}
        
        for future in tqdm(as_completed(future_to_eid), total=len(tasks), desc="XMLダウンロード中"):
            try:
                eid, result_path = future.result()
                # 元のタスク情報に結果（ファイルパス）を追記
                task_info = next(item for item in tasks if item["citing_paper_eid"] == eid)
                task_info['fulltext_xml_path'] = result_path
                updated_tasks.append(task_info)
            except Exception as e:
                eid_for_error = future_to_eid[future]
                print(f"タスク処理中にエラーが発生 (EID: {eid_for_error}): {e}")


# --- 4. 最終的なデータの保存 ---
if updated_tasks:
    final_df = pd.DataFrame(updated_tasks)
    final_df.to_csv(OUTPUT_CITING_PAPERS_CSV, index=False, encoding='utf-8-sig')
    print(f"\n処理完了。XMLのパス情報を更新した {len(final_df)} 件の論文情報を '{OUTPUT_CITING_PAPERS_CSV}' に保存しました。")
    print("\n--- 保存されたデータの先頭5件 ---")
    print(final_df.head())
else:
    print("\n収集できた被引用論文はありませんでした。")

処理対象のデータ論文（被引用数10以上）: 4320件

[ステップA] ダウンロード対象の被引用論文リストを作成しています...


データ論文をスキャン中:   0%|          | 0/4320 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
import requests
import pandas as pd
import time
import os
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- 設定項目 ---
API_KEY = "90469972beed34fa2913dc1ad6a644ac" 
INPUT_CITING_PAPERS_CSV = '../data/processed/citing_papers_raw.csv'
OUTPUT_WITH_PATHS_CSV = '../data/processed/citing_papers_with_paths.csv'
XML_OUTPUT_DIR = '../data/raw/fulltext/'
FULLTEXT_API_URL = "https://api.elsevier.com/content/article/doi/"

# --- 1. 入力データの読み込みと準備 ---
try:
    df_citing_papers = pd.read_csv(INPUT_CITING_PAPERS_CSV)
    df_target = df_citing_papers.dropna(subset=['citing_paper_doi']).copy()
    print(f"'{INPUT_CITING_PAPERS_CSV}' を正常に読み込みました。")
    print(f"合計 {len(df_target)} 件の論文のXMLダウンロードを開始します。")
except FileNotFoundError:
    print(f"エラー: '{INPUT_CITING_PAPERS_CSV}' が見つかりません。")
    df_target = pd.DataFrame()

# --- 【修正点①】ファイル名から不正な文字を取り除く関数を定義 ---
def sanitize_filename(filename):
    """ファイル名として使えない文字をアンダースコアに置換する"""
    invalid_chars = '<>:"/\\|?*'
    for char in invalid_chars:
        filename = filename.replace(char, '_')
    return filename

# --- 2. XMLダウンロード用の関数を定義 ---
def download_xml_by_doi(task):
    doi = task.get('citing_paper_doi')
    if not doi or pd.isna(doi):
        task['fulltext_xml_path'] = None
        task['download_status'] = "failed (DOI is missing)"
        return task

    # 【修正点②】サニタイズ関数を使って安全なファイル名を作成
    safe_filename = sanitize_filename(doi) + '.xml'
    xml_path = os.path.join(XML_OUTPUT_DIR, safe_filename)
    
    if os.path.exists(xml_path):
        task['fulltext_xml_path'] = xml_path
        task['download_status'] = "success (cached)"
        return task

    try:
        url = f"{FULLTEXT_API_URL}{doi}?apiKey={API_KEY}"
        response = requests.get(url, headers={'Accept': 'application/xml'}, timeout=60)
        
        if response.status_code == 200:
            with open(xml_path, 'w', encoding='utf-8') as f:
                f.write(response.text)
            task['fulltext_xml_path'] = xml_path
            task['download_status'] = "success (downloaded)"
        else:
            task['fulltext_xml_path'] = None
            task['download_status'] = f"failed (Status: {response.status_code})"
            
    except requests.exceptions.RequestException as e:
        task['fulltext_xml_path'] = None
        task['download_status'] = "failed (Request Error)"
        
    return task

# --- 3. 並列処理でダウンロードを実行 ---
# (このセクションは変更ありません)
results_list = []
if not df_target.empty:
    os.makedirs(XML_OUTPUT_DIR, exist_ok=True)
    tasks = df_target.to_dict('records')
    
    with ThreadPoolExecutor(max_workers=10) as executor:
        results_list = list(tqdm(executor.map(download_xml_by_doi, tasks), total=len(tasks), desc="XMLダウンロード中"))

# --- 4. 結果の集計と保存 ---
# (このセクションは変更ありません)
if results_list:
    df_results = pd.DataFrame(results_list)
    df_results.to_csv(OUTPUT_WITH_PATHS_CSV, index=False, encoding='utf-8-sig')
    print(f"\n処理完了。結果を '{OUTPUT_WITH_PATHS_CSV}' に保存しました。")
    print("\n--- 処理結果サマリー ---")
    print(df_results['download_status'].value_counts())
else:
    print("処理対象のデータがありませんでした。")

'../data/processed/citing_papers_raw.csv' を正常に読み込みました。
合計 148581 件の論文のXMLダウンロードを開始します。


XMLダウンロード中:   0%|          | 0/148581 [00:00<?, ?it/s]


処理完了。結果を '../data/processed/citing_papers_with_paths.csv' に保存しました。

--- 処理結果サマリー ---
download_status
failed (Status: 429)    90613
failed (Status: 404)    32862
success (cached)        17023
success (downloaded)     8083
Name: count, dtype: int64


429 の再試行


In [ ]:
import requests
import pandas as pd
import time
import os
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# --- 基本設定（前回と同じ） ---
API_KEY = "90469972beed34fa2913dc1ad6a644ac" 
RESULTS_CSV_PATH = '../data/processed/citing_papers_with_paths.csv'
XML_OUTPUT_DIR = '../data/raw/fulltext/'
FULLTEXT_API_URL = "https://api.elsevier.com/content/article/doi/"

# download関数は前回と同じものを使用します
def download_xml_by_doi_with_retry(task, max_retries=3):
    doi = task.get('citing_paper_doi')
    if not doi or pd.isna(doi):
        task['fulltext_xml_path'] = None
        task['download_status'] = "failed (DOI is missing)"
        return task
    safe_filename = doi.replace('/', '_') + '.xml'
    xml_path = os.path.join(XML_OUTPUT_DIR, safe_filename)
    if os.path.exists(xml_path):
        task['fulltext_xml_path'] = xml_path
        task['download_status'] = "success (cached)"
        return task
    for attempt in range(max_retries):
        try:
            url = f"{FULLTEXT_API_URL}{doi}?apiKey={API_KEY}"
            response = requests.get(url, headers={'Accept': 'application/xml'}, timeout=60)
            if response.status_code == 200:
                with open(xml_path, 'w', encoding='utf-8') as f:
                    f.write(response.text)
                task['fulltext_xml_path'] = xml_path
                task['download_status'] = f"success (retry attempt {attempt + 1})"
                return task
            elif response.status_code == 429:
                wait_time = (2 ** attempt) + random.uniform(0, 1)
                time.sleep(wait_time)
            else:
                task['fulltext_xml_path'] = None
                task['download_status'] = f"failed (Status: {response.status_code})"
                return task
        except requests.exceptions.RequestException:
            time.sleep(2 ** attempt)
    task['fulltext_xml_path'] = None
    task['download_status'] = f"failed (retries exhausted)"
    return task

# --- 1. 失敗したタスクの読み込み ---
try:
    df_results = pd.read_csv(RESULTS_CSV_PATH)
    # 【重要】429エラーで失敗したタスクのみを抽出
    retry_targets_df = df_results[df_results['download_status'] == 'failed (Status: 429)'].copy()
    
    if not retry_targets_df.empty:
        print(f"レートリミットで失敗した {len(retry_targets_df)} 件のダウンロードを再試行します。")
        tasks_to_retry = retry_targets_df.to_dict('records')
    else:
        print("429エラーで失敗したタスクはありませんでした。")
        tasks_to_retry = []
        
except FileNotFoundError:
    print(f"エラー: '{RESULTS_CSV_PATH}' が見つかりません。")
    tasks_to_retry = []

# --- 2. 失敗したタスクのみを、より低速で再実行 ---
retry_results_list = []
if tasks_to_retry:
    # 【重要】max_workersの数を大幅に減らして、APIに優しくする
    with ThreadPoolExecutor(max_workers=2) as executor:
        retry_results_list = list(tqdm(executor.map(download_xml_by_doi_with_retry, tasks_to_retry), total=len(tasks_to_retry), desc="429エラー再試行中"))

# --- 3. 元のデータと結果をマージして更新 ---
if retry_results_list:
    # 再試行の結果をDataFrameに変換
    df_retry_results = pd.DataFrame(retry_results_list)
    
    # 元のDataFrameを更新するために、EIDまたはDOIをインデックスに設定
    df_results.set_index('citing_paper_doi', inplace=True)
    df_retry_results.set_index('citing_paper_doi', inplace=True)
    
    # 再試行の結果で元のデータを更新
    df_results.update(df_retry_results)
    
    # インデックスをリセットして元の形式に戻す
    df_results.reset_index(inplace=True)
    
    # 更新されたDataFrameを同じファイルに上書き保存
    df_results.to_csv(RESULTS_CSV_PATH, index=False, encoding='utf-8-sig')
    
    print(f"\n再試行完了。'{RESULTS_CSV_PATH}' を更新しました。")
    print("\n--- 最新の処理結果サマリー ---")
    print(df_results['download_status'].value_counts())
else:
    print("再試行するタスクはありませんでした。")